# Calculate Overflow Per Crossing

Given calculation of capacity, peak flow, and overflow on a point-by-point basis, calculate overflow on a crossing-by-crossing basis *within the existing data model*.

In [2]:
import json
from copy import deepcopy
from collections import OrderedDict
import petl as etl

In [3]:
sample_drainit_config = r'D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\test_results\test_analytics0\drainit_config.json'

In [4]:
with open(sample_drainit_config) as fp:
    j = json.load(fp)

In [5]:
pts = j['points']

In [6]:
shed_field_map = OrderedDict({
    "area_sqkm": "shed_area_sqkm", 
    "avg_slope_pct": "shed_avg_slope_pct", 
    "avg_cn": "shed_avg_cn", 
    "max_fl": "shed_max_fl", 
    "tc_hr": "shed_tc_hr", 
    #"avg_rainfall": "shed_avg_rainfall"
})

FREQUENCIES = [1,2,5,10,25,50,100,200,500,1000]
frequency_fields = [f'y{freq}' for freq in FREQUENCIES]

In [7]:
analytics_field_map = OrderedDict({
    'culvert_peakflow_m3s': 'ppf_m3s',
    'crossing_peakflow_m3s': 'xpf_m3s',
    'culvert_overflow_m3s': 'pof_m3s',
    'crossing_overflow_m3s': 'xof_m3s'
})

In [8]:
capacity_field_map = OrderedDict({
    "culv_mat": "culv_mat",
    "in_type": "in_type",
    "in_shape": "in_shape",
    "in_a": "in_a",
    "in_b": "in_b",
    "hw": "hw",
    "slope": "slope",
    "length": "length",
    "out_shape": "out_shape",
    "out_a": "out_a",
    "out_b": "out_b",
    "crossing_type": "crossing_type",
    "culvert_area_sqm": "culvert_area_sqm",
    "culvert_depth_m": "culvert_depth_m",
    "coefficient_c": "coefficient_c",
    "coefficient_y": "coefficient_y",
    "coefficient_slope": "coefficient_slope",
    "slope_rr": "slope_rr",
    "head_over_invert": "head_over_invert",
    "culvert_capacity": "culvert_capacity",
    "crossing_capacity": "crossing_capacity"
})

In [18]:
pts_table = etl\
    .fromdicts(pts)\
    .addrownumbers(field='oid')\
    .cutout(*['naacc', 'raw', 'notes'])\
    .convert('validation_errors', lambda d: "; ".join(['{0} ({1})'.format(k, ",".join([i for i in v])) for k,v in d.items()]))\
    .unpackdict('capacity', keys=list(capacity_field_map.keys()))\
    .unpackdict('shed', keys=list(shed_field_map.keys()))\
    .rename(shed_field_map)\
    .unpack('analytics', frequency_fields)

pts_table

oid,uid,lat,lng,spatial_ref_code,group_id,failure_event,include,validation_errors,culv_mat,in_type,in_shape,in_a,in_b,hw,slope,length,out_shape,out_a,out_b,crossing_type,culvert_area_sqm,culvert_depth_m,coefficient_c,coefficient_y,coefficient_slope,slope_rr,head_over_invert,culvert_capacity,crossing_capacity,shed_area_sqkm,shed_avg_slope_pct,shed_avg_cn,shed_max_fl,shed_tc_hr,y1,y2,y5,y10,y25,y50,y100,y200,y500,y1000
1,None,42.16342544555664,-73.60004425048828,4326,66697,None,False,Slope_Percent (Field may not be null.); Inlet_Structure_Type (Field may not be null.); Naacc_Culvert_Id (Field may not be null.); Material (Field may not be null.); Crossing_Structure_Length (Field may not be null.); Outlet_Structure_Type (Field may not be null.); Inlet_Width (Field may not be null.); Inlet_Type (Field may not be null.); Outlet_Width (Field may not be null.); Outlet_Height (Field may not be null.); Inlet_Height (Field may not be null.); in_a (must be a number (False)); in_b (must be a number (False)); hw (must be a number (True)); length (must be a number (False)),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,64662,42.102073669433594,-73.54853820800781,4326,66742,None,False,in_shape (Wrong bridge type (Box)); in_a (Bridge wider than 20 ft (64.5 ft)),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,72498,42.30765914916992,-73.7044448852539,4326,75158,None,True,,Concrete,Projecting,Box,2.4384000301361084,1.920240044593811,0.30480000376701355,0.8999999761581421,7.619999885559082,Box Culvert,8.0,6.300000190734863,None,4.682313442230225,1.920240044593811,0.03779999911785126,0.8700000047683716,-0.5,0.008999999612569809,2.2250399589538574,9.978796751827035,19.966097531717068,19.692527509295594,5.899276006604845,66.4802489521147,9958.32421875,1.158055242469295,"{'frequency': '1', 'duration': '24hr', 'avg_rainfall_cm': 5.797259801854439, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 10.185285844829371, 'crossing_peakflow_m3s': 10.185285844829371}, 'overflow': {'culvert_overflow_m3s': -0.20648909300233598, 'crossing_overflow_m3s': 9.780811686887697}}","{'frequency': '2', 'duration': '24hr', 'avg_rainfall_cm': 7.14530681061857, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 24.088545120432766, 'crossing_peakflow_m3s': 24.088545120432766}, 'overflow': {'culvert_overflow_m3s': -14.10974836860573, 'crossing_overflow_m3s': -4.122447588715698}}","{'frequency': '5', 'duration': '24hr', 'avg_rainfall_cm': 9.347852986155235, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 56.02694428100117, 'crossing_peakflow_m3s': 56.02694428100117}, 'overflow': {'culvert_overflow_m3s': -46.04814752917414, 'crossing_overflow_m3s': -36.0608467492841}}","{'frequency': '10', 'duration': '24hr', 'avg_rainfall_cm': 11.174809526228879, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 88.21216224343625, 'crossing_peakflow_m3s': 88.21216224343625}, 'overflow': {'culvert_overflow_m3s': -78.23336549160922, 'crossing_overflow_m3s': -68.24606471171919}}","{'frequency': '25', 'duration': '24hr', 'avg_rainfall_cm': 13.689568901308279, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 137.74172613250914, 'crossing_peakflow_m3s': 137.74172613250914}, 'overflow': {'culvert_overflow_m3s': -127.76292938068211, 'crossing_overflow_m3s': -117.77562860079207}}","{'frequency': '50', 'duration': '24hr', 'avg_rainfall_cm': 15.557819718023612, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 177.17442594104503, 'crossing_peakflow_m3s': 177.17442594104503}, 'overflow': {'culvert_overflow_m3s': -167.19562918921798, 'cros

In [31]:
pts_table2 = deepcopy(pts_table)

for ff in frequency_fields:
    analytics_table = etl\
        .cut(pts_table2, ['oid', ff])\
        .convert(ff, lambda d: dict(**d['overflow'], **d['peakflow']))\
        .unpackdict(ff, list(analytics_field_map.keys()))\
        .prefixheader(f'{ff}_')\
        .rename(f'{ff}_oid', 'oid')
    pts_table2 = etl.join(pts_table2, analytics_table, 'oid')

pts_table2

oid,uid,lat,lng,spatial_ref_code,group_id,failure_event,include,validation_errors,culv_mat,in_type,in_shape,in_a,in_b,hw,slope,length,out_shape,out_a,out_b,crossing_type,culvert_area_sqm,culvert_depth_m,coefficient_c,coefficient_y,coefficient_slope,slope_rr,head_over_invert,culvert_capacity,crossing_capacity,shed_area_sqkm,shed_avg_slope_pct,shed_avg_cn,shed_max_fl,shed_tc_hr,y1,y2,y5,y10,y25,y50,y100,y200,y500,y1000,y1_culvert_peakflow_m3s,y1_crossing_peakflow_m3s,y1_culvert_overflow_m3s,y1_crossing_overflow_m3s,y2_culvert_peakflow_m3s,y2_crossing_peakflow_m3s,y2_culvert_overflow_m3s,y2_crossing_overflow_m3s,y5_culvert_peakflow_m3s,y5_crossing_peakflow_m3s,y5_culvert_overflow_m3s,y5_crossing_overflow_m3s,y10_culvert_peakflow_m3s,y10_crossing_peakflow_m3s,y10_culvert_overflow_m3s,y10_crossing_overflow_m3s,y25_culvert_peakflow_m3s,y25_crossing_peakflow_m3s,y25_culvert_overflow_m3s,y25_crossing_overflow_m3s,y50_culvert_peakflow_m3s,y50_crossing_peakflow_m3s,y50_culvert_overflow_m3s,y50_crossing_overflow_m3s,y100_culvert_peakflow_m3s,y100_crossing_peakflow_m3s,y100_culvert_overflow_m3s,y100_crossing_overflow_m3s,y200_culvert_peakflow_m3s,y200_crossing_peakflow_m3s,y200_culvert_overflow_m3s,y200_crossing_overflow_m3s,y500_culvert_peakflow_m3s,y500_crossing_peakflow_m3s,y500_culvert_overflow_m3s,y500_crossing_overflow_m3s,y1000_culvert_peakflow_m3s,y1000_crossing_peakflow_m3s,y1000_culvert_overflow_m3s,y1000_crossing_overflow_m3s
1,None,42.16342544555664,-73.60004425048828,4326,66697,None,False,Slope_Percent (Field may not be null.); Inlet_Structure_Type (Field may not be null.); Naacc_Culvert_Id (Field may not be null.); Material (Field may not be null.); Crossing_Structure_Length (Field may not be null.); Outlet_Structure_Type (Field may not be null.); Inlet_Width (Field may not be null.); Inlet_Type (Field may not be null.); Outlet_Width (Field may not be null.); Outlet_Height (Field may not be null.); Inlet_Height (Field may not be null.); in_a (must be a number (False)); in_b (must be a number (False)); hw (must be a number (True)); length (must be a number (False)),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,64662,42.102073669433594,-73.54853820800781,4326,66742,None,False,in_shape (Wrong bridge type (Box)); in_a (Bridge wider than 20 ft (64.5 ft)),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,72498,42.30765914916992,-73.7044448852539,4326,75158,None,True,,Concrete,Projecting,Box,2.4384000301361084,1.920240044593811,0.30480000376701355,0.8999999761581421,7.619999885559082,Box Culvert,8.0,6.300000190734863,None,4.682313442230225,1.920240044593811,0.03779999911785126,0.8700000047683716,-0.5,0.008999999612569809,2.2250399589538574,9.978796751827035,19.966097531717068,19.692527509295594,5.899276006604845,66.4802489521147,9958.32421875,1.158055242469295,"{'frequency': '1', 'duration': '24hr', 'avg_rainfall_cm': 5.797259801854439, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 10.185285844829371, 'crossing_peakflow_m3s': 10.185285844829371}, 'overflow': {'culvert_overflow_m3s': -0.20648909300233598, 'crossing_overflow_m3s': 9.780811686887697}}","{'frequency': '2', 'duration': '24hr', 'avg_rainfall_cm': 7.14530681061857, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'culvert_peakflow_m3s': 24.088545120432766, 'crossing_pe

In [34]:
h = etl.header(pts_table2)
print(h, len(h))

('oid', 'uid', 'lat', 'lng', 'spatial_ref_code', 'group_id', 'failure_event', 'include', 'validation_errors', 'culv_mat', 'in_type', 'in_shape', 'in_a', 'in_b', 'hw', 'slope', 'length', 'out_shape', 'out_a', 'out_b', 'crossing_type', 'culvert_area_sqm', 'culvert_depth_m', 'coefficient_c', 'coefficient_y', 'coefficient_slope', 'slope_rr', 'head_over_invert', 'culvert_capacity', 'crossing_capacity', 'shed_area_sqkm', 'shed_avg_slope_pct', 'shed_avg_cn', 'shed_max_fl', 'shed_tc_hr', 'y1', 'y2', 'y5', 'y10', 'y25', 'y50', 'y100', 'y200', 'y500', 'y1000', 'y1_culvert_peakflow_m3s', 'y1_crossing_peakflow_m3s', 'y1_culvert_overflow_m3s', 'y1_crossing_overflow_m3s', 'y2_culvert_peakflow_m3s', 'y2_crossing_peakflow_m3s', 'y2_culvert_overflow_m3s', 'y2_crossing_overflow_m3s', 'y5_culvert_peakflow_m3s', 'y5_crossing_peakflow_m3s', 'y5_culvert_overflow_m3s', 'y5_crossing_overflow_m3s', 'y10_culvert_peakflow_m3s', 'y10_crossing_peakflow_m3s', 'y10_culvert_overflow_m3s', 'y10_crossing_overflow_m3s',

In [39]:
def find_max_return_period(row):
    # convert row to dict
    r = {k: v for k, v in zip(list(h), row)}
    # assemble a list of all crossing overflows, ordered by frequency
    ovfs = [f for f in [r.get(f'{ff}_crossing_overflow_m3s', None) for ff in frequency_fields] if f]
    # filter that list to just those that don't overflow
    handled_return_periods_idx = [ovfs.index(o) for o in ovfs if o >= 0]
    # get the biggest event that doesn't overflow, and return it
    if len(handled_return_periods_idx) > 0:
        return FREQUENCIES[max(handled_return_periods_idx)]
    return None
    
pts_table3 = etl\
    .convert(pts_table2, 'failure_event', lambda v, row: find_max_return_period(row), pass_row=True, failonerror=True)\
    .cutout(*frequency_fields)\

pts_table3

oid,uid,lat,lng,spatial_ref_code,group_id,failure_event,include,validation_errors,culv_mat,in_type,in_shape,in_a,in_b,hw,slope,length,out_shape,out_a,out_b,crossing_type,culvert_area_sqm,culvert_depth_m,coefficient_c,coefficient_y,coefficient_slope,slope_rr,head_over_invert,culvert_capacity,crossing_capacity,shed_area_sqkm,shed_avg_slope_pct,shed_avg_cn,shed_max_fl,shed_tc_hr,y1_culvert_peakflow_m3s,y1_crossing_peakflow_m3s,y1_culvert_overflow_m3s,y1_crossing_overflow_m3s,y2_culvert_peakflow_m3s,y2_crossing_peakflow_m3s,y2_culvert_overflow_m3s,y2_crossing_overflow_m3s,y5_culvert_peakflow_m3s,y5_crossing_peakflow_m3s,y5_culvert_overflow_m3s,y5_crossing_overflow_m3s,y10_culvert_peakflow_m3s,y10_crossing_peakflow_m3s,y10_culvert_overflow_m3s,y10_crossing_overflow_m3s,y25_culvert_peakflow_m3s,y25_crossing_peakflow_m3s,y25_culvert_overflow_m3s,y25_crossing_overflow_m3s,y50_culvert_peakflow_m3s,y50_crossing_peakflow_m3s,y50_culvert_overflow_m3s,y50_crossing_overflow_m3s,y100_culvert_peakflow_m3s,y100_crossing_peakflow_m3s,y100_culvert_overflow_m3s,y100_crossing_overflow_m3s,y200_culvert_peakflow_m3s,y200_crossing_peakflow_m3s,y200_culvert_overflow_m3s,y200_crossing_overflow_m3s,y500_culvert_peakflow_m3s,y500_crossing_peakflow_m3s,y500_culvert_overflow_m3s,y500_crossing_overflow_m3s,y1000_culvert_peakflow_m3s,y1000_crossing_peakflow_m3s,y1000_culvert_overflow_m3s,y1000_crossing_overflow_m3s
1,None,42.16342544555664,-73.60004425048828,4326,66697,None,False,Slope_Percent (Field may not be null.); Inlet_Structure_Type (Field may not be null.); Naacc_Culvert_Id (Field may not be null.); Material (Field may not be null.); Crossing_Structure_Length (Field may not be null.); Outlet_Structure_Type (Field may not be null.); Inlet_Width (Field may not be null.); Inlet_Type (Field may not be null.); Outlet_Width (Field may not be null.); Outlet_Height (Field may not be null.); Inlet_Height (Field may not be null.); in_a (must be a number (False)); in_b (must be a number (False)); hw (must be a number (True)); length (must be a number (False)),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,64662,42.102073669433594,-73.54853820800781,4326,66742,None,False,in_shape (Wrong bridge type (Box)); in_a (Bridge wider than 20 ft (64.5 ft)),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,72498,42.30765914916992,-73.7044448852539,4326,75158,1,True,,Concrete,Projecting,Box,2.4384000301361084,1.920240044593811,0.30480000376701355,0.8999999761581421,7.619999885559082,Box Culvert,8.0,6.300000190734863,None,4.682313442230225,1.920240044593811,0.03779999911785126,0.8700000047683716,-0.5,0.008999999612569809,2.2250399589538574,9.978796751827035,19.966097531717068,19.692527509295594,5.899276006604845,66.4802489521147,9958.32421875,1.158055242469295,10.185285844829371,10.185285844829371,-0.20648909300233598,9.780811686887697,24.088545120432766,24.088545120432766,-14.10974836860573,-4.122447588715698,56.02694428100117,56.02694428100117,-46.04814752917414,-36.0608467492841,88.21216224343625,88.21216224343625,-78.23336549160922,-68.24606471171919,137.74172613250914,137.74172613250914,-127.76292938068211,-117.77562860079207,177.17442594104503,177.17442594104503,-167.19562918921798,-157.20832840932798,221.34350478117034,221.34350478117034,-211.3647080293433,-201.37740724945326,276.01002146528396,276.01002146528396,-266.03122471345694,-256.0439239335669,361.03581592370125,361.03581592370125,-351.05701917187423,-34

In [59]:
len(etl.header(pts_table2))

76

In [71]:
field_types_lookup = {}
for h in etl.header(pts_table2):
    ftypes = [n for n in etl.typeset(pts_table2, h) if n != 'NoneType']
    if 'float' in ftypes:
        field_types_lookup[h] = float
    elif 'int' in ftypes:
        field_types_lookup[h] = int
    elif 'str' in ftypes:
        field_types_lookup[h] = str
    else:
        field_types_lookup[h] = str